In [2]:
import utilities.light_generator as ulg
import msprime


MU_HUMAN = ulg.give_mu()
RHO_HUMAN = ulg.give_mu() * ulg.give_random_coeff()
POPULATION = 100
num_replicates = 1
lengt = int(3e9)
is_experement = True

events = [
    msprime.PopulationParametersChange(0, initial_size=POPULATION, growth_rate=0),
    msprime.PopulationParametersChange(10, 80),
    msprime.PopulationParametersChange(50, 120),
    msprime.PopulationParametersChange(100, 1),
    #msprime.PopulationParametersChange(0.1, 4000, growth_rate=0),
    #msprime.PopulationParametersChange(1, 6000, growth_rate=0)
]

ev = demographic_events=ulg.generate_demographic_events(POPULATION)

dg = ulg.DataGenerator(
    recombination_rate=RHO_HUMAN,
    mutation_rate=MU_HUMAN,
    sample_size=100,
    #demographic_events=None,
    #demographic_events=events,
    demographic_events=ulg.generate_demographic_events(POPULATION),
    num_replicates=num_replicates,
    lengt=lengt,
    random_seed=42,
    is_experement=is_experement)
dg.run_simulation()

<generator object _replicate_generator at 0x13ff82f90>

In [2]:
import numpy as np

In [3]:
def give_random_coeff(mean=.128, var=.05):
    return np.random.normal(.128,.005)

In [4]:
ulg.give_mu()

7.81824851441265e-07

In [8]:
ulg.RHO_LIMIT

(-117.95067711470662, 82.04932288529338)

In [64]:
RHO_HUMAN/MU_HUMAN

0.128

In [5]:
ev = demographic_events=ulg.generate_demographic_events(POPULATION)
dd = msprime.DemographyDebugger(demographic_events=ev)
dd.print_history()

Model =  hudson(reference_size=1)
Events @ generation 0
   - Population parameter change for -1: initial_size -> 14 growth_rate -> 0 
Epoch: 0 -- 0.0 generations
     start     end      growth_rate |     0    
   -------- --------       -------- | -------- 
0 |    1        1                 0 |     0    

Epoch: 0.0 -- 0.012078290427782523 generations
     start     end      growth_rate |     0    
   -------- --------       -------- | -------- 
0 |   14       14                 0 |     0    

Events @ generation 0.012078290427782523
   - Population parameter change for -1: initial_size -> 30 growth_rate -> 0 
Epoch: 0.012078290427782523 -- 0.013433172995274356 generations
     start     end      growth_rate |     0    
   -------- --------       -------- | -------- 
0 |   30       30                 0 |     0    

Events @ generation 0.013433172995274356
   - Population parameter change for -1: initial_size -> 17 growth_rate -> 0 
Epoch: 0.013433172995274356 -- 0.16526486507700566 gen

In [4]:
%%time
data = next(dg)

CPU times: user 6.47 s, sys: 817 ms, total: 7.28 s
Wall time: 7.29 s


3000000000

In [34]:
import math

def out_of_africa():
    # First we set out the maximum likelihood values of the various parameters
    # given in Table 1.
    N_A = 7300
    N_B = 2100
    N_AF = 12300
    N_EU0 = 1000
    N_AS0 = 510
    # Times are provided in years, so we convert into generations.
    generation_time = 25
    T_AF = 220e3 / generation_time
    T_B = 140e3 / generation_time
    T_EU_AS = 21.2e3 / generation_time
    # We need to work out the starting (diploid) population sizes based on
    # the growth rates provided for these two populations
    r_EU = 0.004
    r_AS = 0.0055
    N_EU = N_EU0 / math.exp(-r_EU * T_EU_AS)
    N_AS = N_AS0 / math.exp(-r_AS * T_EU_AS)
    # Migration rates during the various epochs.
    m_AF_B = 25e-5
    m_AF_EU = 3e-5
    m_AF_AS = 1.9e-5
    m_EU_AS = 9.6e-5
    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU and 2=CHB
    # initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=0, initial_size=N_AF),
        msprime.PopulationConfiguration(
            sample_size=1, initial_size=N_EU, growth_rate=r_EU),
        msprime.PopulationConfiguration(
            sample_size=1, initial_size=N_AS, growth_rate=r_AS)
    ]
    migration_matrix = [
        [      0, m_AF_EU, m_AF_AS],
        [m_AF_EU,       0, m_EU_AS],
        [m_AF_AS, m_EU_AS,       0],
    ]
    demographic_events = [
        # CEU and CHB merge into B with rate changes at T_EU_AS
        msprime.MassMigration(
            time=T_EU_AS, source=2, destination=1, proportion=1.0),
        msprime.MigrationRateChange(time=T_EU_AS, rate=0),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0, population_id=1),
        # Population B merges into YRI at T_B
        msprime.MassMigration(
            time=T_B, source=1, destination=0, proportion=1.0),
        msprime.MigrationRateChange(time=T_B, rate=0),
        # Size changes to N_A at T_AF
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]
    # Use the demography debugger to print out the demographic history
    # that we have just described.
    dd = msprime.DemographyDebugger(
        population_configurations=population_configurations,
        migration_matrix=migration_matrix,
        demographic_events=demographic_events)
    dd.print_history()
    return 

In [35]:
out_of_africa()

Model =  hudson(reference_size=1)
Epoch: 0 -- 848.0 generations
     start     end      growth_rate |     0        1        2    
   -------- --------       -------- | -------- -------- -------- 
0 |1.23e+04 1.23e+04              0 |     0      3e-05   1.9e-05 
1 |2.97e+04   1e+03           0.004 |   3e-05      0     9.6e-05 
2 |5.41e+04    510           0.0055 |  1.9e-05  9.6e-05     0    

Events @ generation 848.0
   - Mass migration: Lineages moved with probability 1.0 backwards in time with source 2 & dest 1
                     (equivalent to migration from 1 to 2 forwards in time)
   - Migration rate change to 0 everywhere
   - Migration rate change for (0, 1) to 0.00025
   - Migration rate change for (1, 0) to 0.00025
   - Population parameter change for 1: initial_size -> 2100 growth_rate -> 0 
Epoch: 848.0 -- 5600.0 generations
     start     end      growth_rate |     0        1        2    
   -------- --------       -------- | -------- -------- -------- 
0 |1.23e+04 1.23e+